In [ ]:
import os
import SimpleITK as sitk
import numpy as np
import nibabel as nib
from mayavi import mlab
import vtk

# Function finds correct files
def find_original_files(folder_path):
    airway_file = None
    lung_file = None
    
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith("airtree-recentered.nii.gz"):
                airway_file = os.path.join(root, file)
            elif file.endswith("lung-recentered.nii.gz"):
                lung_file = os.path.join(root, file)
        
        if airway_file and lung_file:
            break  # All files are found, exit the loop
    
    return airway_file, lung_file


def find_vtk_files(folder_path):
    airway_file = None
    lung_file = None
    
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith("airtree-recentered_output.vtk"):
                airway_file = os.path.join(root, file)
            elif file.endswith("lung-recentered_output.vtk"):
                lung_file = os.path.join(root, file)
        
        if airway_file and lung_file:
            break  # All files are found, exit the loop
    
    return airway_file, lung_file

# Define the output directory where you want to save the new files
output_directory = r"Z:\Q- Figures\T32-Poster\Allyson\Experiment\Case 24"

# Find correct files in base folder
base_folder = r"z:\E-Kelly-VidaSegmentals&Sublobes\2- 2AdditionalScans\Kelly Done\SPIROMICS-51287502\A1-Recentered"

airway_file, lung_file = find_original_files(base_folder)

if airway_file and lung_file:
    print("All files were found for", base_folder)
    print("Airway File:", airway_file)
    print("Lung File:", lung_file)

# Define a list of file paths for your files (airway, lung, artery)
file_paths = [airway_file, lung_file]

# Loop through the file paths
for nii_file in file_paths:
    # Load the NIfTI (.nii.gz) file
    image = sitk.ReadImage(nii_file)

    # Convert the SimpleITK image to a NumPy array
    image_array = sitk.GetArrayFromImage(image)

    # Define the output NRRD file name
    nrrd_file = os.path.join(output_directory, os.path.basename(nii_file).replace(".nii.gz", ".nrrd"))

    # Convert and save the NumPy array to an NRRD file
    sitk.WriteImage(sitk.GetImageFromArray(image_array), nrrd_file)

    # Load the .nrrd segmentation for Vseed
    nrrd_reader_vseed = vtk.vtkNrrdReader()
    nrrd_reader_vseed.SetFileName(nrrd_file)
    nrrd_reader_vseed.Update()
    vtk_image_data_vseed = vtk.vtkImageData()
    vtk_image_data_vseed.DeepCopy(nrrd_reader_vseed.GetOutput())

    # Define scalar value for "Vseed" segment
    vseed_scalar_value = 2

    # Create a Marching Cubes filter for "Vseed" segment
    contour_vseed = vtk.vtkMarchingCubes()
    contour_vseed.SetInputData(vtk_image_data_vseed)
    contour_vseed.SetValue(0, vseed_scalar_value)
    contour_vseed.Update()

    # Define the output VTK file name
    output_vtk_file = os.path.join(output_directory, os.path.basename(nii_file).replace(".nii.gz", "_output.vtk"))

    # Write the "Vseed" mesh to a .vtk file
    vtk_writer_vseed = vtk.vtkPolyDataWriter()
    vtk_writer_vseed.SetFileName(output_vtk_file)
    vtk_writer_vseed.SetInputData(contour_vseed.GetOutput())
    vtk_writer_vseed.Write()

    # # Specify the path for the decimated VTK file
    # output_vtk_file_path = os.path.join(output_directory, os.path.basename(nii_file).replace(".nii.gz", "_decimated.vtk"))

    # # Create a VTK reader for the original dataset
    # reader = vtk.vtkDataSetReader()
    # reader.SetFileName(output_vtk_file)
    # reader.Update()

    # # Create a vtkDecimatePro filter to decimate the dataset
    # decimate = vtk.vtkDecimatePro()
    # decimate.SetInputConnection(reader.GetOutputPort())
    # decimate.SetTargetReduction(0.9)  # 90% reduction

    # # Create a VTK writer for the decimated dataset
    # writer = vtk.vtkDataSetWriter()
    # writer.SetFileName(output_vtk_file_path)
    # writer.SetInputConnection(decimate.GetOutputPort())
    # writer.Write()

    print(f"Process Complete for {nii_file}")

# Define the paths to your VTK files
vtk_file_A, vtk_file_L = find_vtk_files(output_directory)

mlab.figure(bgcolor=(1, 1, 1))

# Load the VTK files using Mayavi's open function
srcA = mlab.pipeline.open(vtk_file_A)
srcL = mlab.pipeline.open(vtk_file_L)

# Create a surface visualization of the data with colors and opacity
surfaceA = mlab.pipeline.surface(srcA, color=(1, 0.75, 0), opacity=1)
surfaceL = mlab.pipeline.surface(srcL, color=(0.4, 0.5, 0.4), opacity=0.09)

# Create a VTK reader to calculate the center
reader = vtk.vtkDataSetReader()
reader.SetFileName(vtk_file_A)
reader.Update()

# Get the bounds of the loaded VTK dataset
bounds = reader.GetOutput().GetBounds()

# Calculate the center using the bounds
center = [(bounds[0] + bounds[1]) / 2, (bounds[2] + bounds[3]) / 2, (bounds[4] + bounds[5]) / 2]

# Calculate the distance between the camera and the object (adjust as needed)
distance = 2000  # You can adjust this value to change the camera distance

# Calculate the camera position based on the center and distance
camera_position = [center[0], center[1] + distance, center[2]]

# Set the camera parameters
mlab.view(azimuth=90, elevation=90, distance=distance, focalpoint=center)

# Save a screenshot with a magnification factor
magnification_factor = 5
mlab.savefig(os.path.join(output_directory, "screenshot.png"), magnification=magnification_factor)

print("Screenshot saved at:", output_directory)

In [ ]:
import os
import SimpleITK as sitk
import numpy as np
import nibabel as nib
from mayavi import mlab
import vtk
import nrrd

# Function finds correct files
def find_original_files(folder_path):
    airway_file = None
    lung_file = None
    
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith("airtree-recentered.nii.gz"):
                airway_file = os.path.join(root, file)
            elif file.endswith("lung-recentered.nii.gz"):
                lung_file = os.path.join(root, file)
        
        if airway_file and lung_file:
            break  # All files are found, exit the loop
    
    return airway_file, lung_file

def find_nrrd_files(folder_path):
    airway_file = None
    lung_file = None
    
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith("airtree-recentered.nrrd"):
                airway_file = os.path.join(root, file)
            elif file.endswith("lung-recentered.nrrd"):
                lung_file = os.path.join(root, file)
        
        if airway_file and lung_file:
            break  # All files are found, exit the loop
    
    return airway_file, lung_file


def find_vtk_files(folder_path):
    airway_file = None
    lung_file = None
    
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith("airtree-recentered.vtk"):
                airway_file = os.path.join(root, file)
            elif file.endswith("lung-recentered.vtk"):
                lung_file = os.path.join(root, file)
        
        if airway_file and lung_file:
            break  # All files are found, exit the loop
    
    return airway_file, lung_file


# Define the output directory where you want to save the new files
output_directory = r"Z:\Q- Figures\T32-Poster\Allyson\Experiment\Case 24"

# Find correct files in base folder
base_folder = r"z:\E-Kelly-VidaSegmentals&Sublobes\2- 2AdditionalScans\Kelly Done\SPIROMICS-51287502\A1-Recentered"


airway_file, lung_file = find_original_files(base_folder)

if airway_file and lung_file:
    print("All files were found for", base_folder)
    print("Airway File:", airway_file)
    print("Lung File:", lung_file)

# Define a list of file paths for your files (airway, lung, artery)
file_paths = [airway_file, lung_file]

# Loop through the file paths
for nii_file in file_paths:

    nifti_image = sitk.ReadImage(nii_file)

    # Define the output NRRD file name
    nrrd_file = os.path.join(output_directory, os.path.basename(nii_file).replace(".nii.gz", ".nrrd"))

    # Write the NRRD image with the same scale as the NIfTI image
    sitk.WriteImage(nifti_image, nrrd_file)


airway_nrrd_file, lung_nrrd_file = find_nrrd_files(output_directory)

nrrd_file_path = [airway_nrrd_file, lung_nrrd_file]

for nrrd_file in nrrd_file_path:
    # Load the .nrrd segmentation for Vseed
    nrrd_reader_vseed = vtk.vtkNrrdReader()
    nrrd_reader_vseed.SetFileName(nrrd_file)
    nrrd_reader_vseed.Update()
    vtk_image_data_vseed = vtk.vtkImageData()
    vtk_image_data_vseed.DeepCopy(nrrd_reader_vseed.GetOutput())

    # Define scalar value for "Vseed" segment
    vseed_scalar_value = 2

    # Create a Marching Cubes filter for "Vseed" segment
    contour_vseed = vtk.vtkMarchingCubes()
    contour_vseed.SetInputData(vtk_image_data_vseed)
    contour_vseed.SetValue(0, vseed_scalar_value)
    contour_vseed.Update()

    # Define the output VTK file name
    output_vtk_file = os.path.join(output_directory, os.path.basename(nrrd_file).replace(".nrrd", ".vtk"))

    # Write the "Vseed" mesh to a .vtk file
    vtk_writer_vseed = vtk.vtkPolyDataWriter()
    vtk_writer_vseed.SetFileName(output_vtk_file)
    vtk_writer_vseed.SetInputData(contour_vseed.GetOutput())
    vtk_writer_vseed.Write()

    print(f"Process Complete for {nrrd_file}")

# Define the paths to your VTK files
vtk_file_Air, vtk_file_L, vtk_file_AT= find_vtk_files(output_directory)

vtk_file_AT = (r'Z:\Q- Figures\T32-Poster\Nate\Experiment\Case-24\Case-24-0.5__QR40__RESEARCH__M_0.6_Qr40d_3_51287502Aseed.vtk')

mlab.figure(bgcolor=(1, 1, 1))

# Load the VTK files using Mayavi's open function
srcA = mlab.pipeline.open(vtk_file_Air)
srcL = mlab.pipeline.open(vtk_file_L)
srcAT = mlab.pipeline.open(vtk_file_AT)

# Create a surface visualization of the data with colors and opacity
surfaceA = mlab.pipeline.surface(srcA, color=(1, 0.75, 0), opacity=1)
surfaceL = mlab.pipeline.surface(srcL, color=(128/255, 174/255, 128/255), opacity=0.15)
surfaceAT = mlab.pipeline.surface(srcAT, color=(0, 151/255, 206/255), opacity=1)

# Create a VTK reader to calculate the center
reader = vtk.vtkDataSetReader()
reader.SetFileName(vtk_file_L)
reader.Update()

# Get the bounds of the loaded VTK dataset
bounds = reader.GetOutput().GetBounds()

# Calculate the center using the bounds
center = [(bounds[0] + bounds[1]) / 2, (bounds[2] + bounds[3]) / 2, (bounds[4] + bounds[5]) / 2]

# Calculate the distance between the camera and the object (adjust as needed)
distance = 600  # You can adjust this value to change the camera distance

# Calculate the camera position based on the center and distance
camera_position = [center[0], center[1] + distance, center[2]]

# Set the camera parameters
mlab.view(azimuth=90, elevation=90, distance=distance, focalpoint=center)

# Save a screenshot with a magnification factor
magnification_factor = 5
mlab.savefig(os.path.join(output_directory, "screenshot.png"), magnification=magnification_factor)

print("Screenshot saved at:", output_directory)